In [1]:
import torch
from fastbook import *

matplotlib.rc('image', cmap='Greys')

In [2]:
path = untar_data(URLs.MNIST)

In [3]:
Path.BASE_PATH = path

In [4]:
path.ls()

(#2) [Path('training'),Path('testing')]

In [5]:
(path/'training').ls()

(#10) [Path('training/7'),Path('training/8'),Path('training/5'),Path('training/6'),Path('training/9'),Path('training/3'),Path('training/2'),Path('training/0'),Path('training/4'),Path('training/1')]

In [584]:
xs = []
ys = []
for i in range(10):
  xs.append(torch.stack([tensor(Image.open(o)) 
                          for o in (path/'training'/f'{i}').ls() ]).float()/255)
  ys += [i]*len((path/'training'/f'{i}').ls())

In [585]:
train_x = torch.cat(xs).view(-1, 28*28)
train_x.shape

torch.Size([60000, 784])

In [586]:
train_y = tensor(ys)
train_y.shape

torch.Size([60000])

In [587]:
def init_params(size, std=1.):
  return (torch.randn(size)*std).requires_grad_()

In [588]:
w = init_params((28*28, 10))
b = init_params(10)

In [589]:
def linear(x): return x@w + b

In [590]:
acts = linear(train_x[:5])
acts

tensor([[ -1.1250, -14.8991,   5.2417,  -0.3008,   5.7340,  -7.2487,   8.3564,   4.6382,   3.9403, -13.6559],
        [ -8.1837,  -5.0914,  -2.2739,  -7.2089,  19.4128,   5.7646,   8.5149,   3.0100,   2.0764,   1.3862],
        [  8.0232,  -1.4075, -17.4992,  -9.3129,  28.8827,   2.7493,   5.6040,  16.0178,  -3.5600,  -7.9606],
        [  9.1720,  -8.8000,  -4.8951, -28.3459,   7.3776, -23.0282,  15.8965,   8.2482,  11.3917, -13.2118],
        [  4.9161,  -9.0179,   8.1191,  -5.5356,   9.3036,  -8.7133,  13.4189,  11.0129,  -4.3309, -16.2302]], grad_fn=<AddBackward0>)

In [591]:
sm_acts = acts.softmax(1)
yt = train_y[:5]
sm_acts, yt

(tensor([[6.6103e-05, 6.8892e-11, 3.8481e-02, 1.5071e-04, 6.2957e-02, 1.4478e-07, 8.6683e-01, 2.1045e-02, 1.0473e-02, 2.3883e-10],
         [1.0351e-12, 2.2801e-11, 3.8156e-10, 2.7436e-12, 9.9998e-01, 1.1820e-06, 1.8497e-05, 7.5221e-08, 2.9571e-08, 1.4829e-08],
         [8.7263e-10, 7.0011e-14, 7.1880e-21, 2.5816e-17, 1.0000e+00, 4.4713e-12, 7.7659e-11, 2.5873e-06, 8.1341e-15, 9.9810e-17],
         [1.1858e-03, 1.8574e-11, 9.2202e-10, 6.0282e-20, 1.9711e-04, 1.2293e-17, 9.8723e-01, 4.7077e-04, 1.0915e-02, 2.2536e-13],
         [1.8250e-04, 1.6211e-10, 4.4910e-03, 5.2742e-09, 1.4681e-02, 2.1985e-10, 8.9953e-01, 8.1117e-02, 1.7594e-08, 1.1956e-13]], grad_fn=<SoftmaxBackward0>),
 tensor([0, 0, 0, 0, 0]))

In [592]:
len(sm_acts), sm_acts.sum(1)

(5, tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000], grad_fn=<SumBackward1>))

In [593]:
sm_acts[range(5),yt]

tensor([6.6103e-05, 1.0351e-12, 8.7263e-10, 1.1858e-03, 1.8250e-04], grad_fn=<IndexBackward0>)

In [594]:
def log_softmax(preds):
  return torch.log(preds.softmax(1))

In [595]:
-log_softmax(linear(train_x[:5]))[range(5), train_y[:5]]

tensor([ 9.6243, 27.5966, 20.8595,  6.7373,  8.6087], grad_fn=<NegBackward0>)

In [596]:
F.nll_loss(torch.log(linear(train_x[:5]).softmax(1)), train_y[:5])

tensor(14.6853, grad_fn=<NllLossBackward0>)

In [597]:
F.cross_entropy(linear(train_x[:5]), train_y[:5])

tensor(14.6853, grad_fn=<NllLossBackward0>)

In [598]:
torch.nn.CrossEntropyLoss()(linear(train_x[:5]), train_y[:5])

tensor(14.6853, grad_fn=<NllLossBackward0>)

In [599]:
def mnist_loss(preds, acts):
  idx = range(len(acts))
  return -log_softmax(preds)[idx, acts].mean()

In [600]:
mnist_loss(linear(train_x[:5]), train_y[:5])

tensor(14.6853, grad_fn=<NegBackward0>)

In [601]:
dset = list(zip(train_x, train_y))

In [602]:
dsetl = DataLoader(dset, batch_size=256)

In [603]:
def gradient_step(x, y, model):
  preds = model(x)
  loss = mnist_loss(preds, y)
  loss.backward()

In [604]:
def batch_accuracy(preds, y):
  preds = preds.softmax(1)[range(len(y)), y]
  correct = (preds > 0.95)
  return correct.float().mean()

In [605]:
batch_accuracy(linear(train_x[:5]), train_y[:5])

tensor(0.)

In [606]:
class Optim:
  def __init__(self, params, lr) -> None:
    self.params, self.lr = list(params), lr
  
  def step(self):
    for p in self.params:
      p.data -= p.grad.data * self.lr
  
  def zero(self):
    for p in self.params:
      p.grad.zero_()

In [607]:
def train(dls, model, opt, num_epochs=5):
  for i in range(num_epochs):
    for xb, yb in dls:
      gradient_step(xb, yb, model)
      opt.step()
      opt.zero()
    
    print(batch_accuracy(model(xb), yb))

In [608]:
params = w,b
opt = Optim(params, 0.1)

train(dsetl, linear, opt, num_epochs=20)

tensor(0.9271)
tensor(0.9375)
tensor(0.9375)
tensor(0.9271)
tensor(0.9271)
tensor(0.9479)
tensor(0.9479)
tensor(0.9479)
tensor(0.9479)
tensor(0.9479)
tensor(0.9479)
tensor(0.9479)
tensor(0.9479)
tensor(0.9479)
tensor(0.9479)
tensor(0.9479)
tensor(0.9479)
tensor(0.9479)
tensor(0.9375)
tensor(0.9271)


In [609]:
linear(train_x[:5]).softmax(1)[range(5), train_y[:5]]


tensor([0.9999, 1.0000, 1.0000, 0.2881, 1.0000], grad_fn=<IndexBackward0>)

In [610]:
img_path = (path/'training'/'0').ls()[3]
img = Image.open(img_path)
img

In [611]:
test_xs = []
test_ys = []
for i in range(10):
  test_xs.append(torch.stack([tensor(Image.open(o)) 
                          for o in (path/'testing'/f'{i}').ls() ]).float()/255)
  test_ys += [i]*len((path/'testing'/f'{i}').ls())

In [612]:
test_x = torch.cat(test_xs).view(-1, 28*28)
test_y = torch.tensor(test_ys)
test_dset = list(zip(test_x, test_y))

In [613]:
test_dset_loader = DataLoader(test_dset)

In [614]:
batch_accuracy(linear(test_x), test_y)

tensor(0.5245)

In [615]:
linear(test_x[:5]).softmax(1)[range(5), test_y[:5]]

tensor([0.9439, 0.9997, 1.0000, 0.9989, 0.2996], grad_fn=<IndexBackward0>)

In [616]:
img_path = (path/'testing'/'0').ls()[4]
img = Image.open(img_path)
img

In [ ]:
neural_net = torch.nn.Sequential(
  torch.nn.Linear(28*28, 450),
  torch.nn.ReLU(),
  torch.nn.Linear(450, 10),
  torch.nn.Softmax(dim=-1)
)

def nn_batch_accuracy(preds, acts):
  idx = range(len(acts))
  correct = (preds[idx, acts] > 0.95)
  return correct.float().mean()

learner = Learner(DataLoaders(dsetl, test_dset_loader), neural_net, 
                  loss_func=torch.nn.CrossEntropyLoss(), opt_func=SGD,
                  metrics=nn_batch_accuracy)

In [ ]:
learner.fit(20, 8e-2)

epoch,train_loss,valid_loss,nn_batch_accuracy,time
0,2.251955,2.302238,0.027750,00:00
1,2.063459,2.249845,0.084183,00:00
2,1.954836,2.152755,0.095933,00:00
3,1.888305,2.091505,0.127650,00:00
4,1.855887,2.053017,0.166583,00:00
5,1.838477,2.027930,0.212450,00:00
6,1.827112,2.010139,0.249200,00:00
7,1.812435,1.992324,0.280583,00:00
8,1.806601,1.973650,0.314167,00:00
9,1.801617,1.955815,0.339667,00:00


In [ ]:
learner.fit??

Signature:
learner.fit(
    n_epoch,
    lr=None,
    wd=None,
    cbs=None,
    reset_opt=False,
    start_epoch=0,
)
Docstring: Fit `self.model` for `n_epoch` using `cbs`. Optionally `reset_opt`.
Source:   
    def fit(self, n_epoch, lr=None, wd=None, cbs=None, reset_opt=False, start_epoch=0):
        if start_epoch != 0:
            cbs = L(cbs) + SkipToEpoch(start_epoch)
        with self.added_cbs(cbs):
            if reset_opt or not self.opt: self.create_opt()
            if wd is None: wd = self.wd
            if wd is not None: self.opt.set_hypers(wd=wd)
            self.opt.set_hypers(lr=self.lr if lr is None else lr)
            self.n_epoch = n_epoch
            self._with_events(self._do_fit, 'fit', CancelFitException, self._end_cleanup)
File:      ~/opt/anaconda3/envs/fastai/lib/python3.9/site-packages/fastai/learner.py
Type:      method

In [ ]:
digits = DataBlock(
  blocks=(ImageBlock, CategoryBlock),
  get_items=get_image_files,
  splitter=RandomSplitter(seed=42),
  get_y=parent_label,
)
learner = vision_learner(digits.dataloaders(path/'training'), 
                         resnet18, metrics=error_rate)
learner.fine_tune(1, base_lr=.1)

epoch,train_loss,valid_loss,error_rate,time
0,0.088885,0.043944,0.011917,07:55
